In [0]:

%pip install folium
%pip install geopandas
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F  # Потрібно для F.lit()

spark = SparkSession.builder.getOrCreate()

silver_db = "airbnb_lab3.airbnb_silver"

# --- 1. Шляхи до Silver-таблиць ---
table_la = f"{silver_db}.silver_listings_la"
table_ny = f"{silver_db}.silver_listings_nyc"
https://dbc-9664db5c-62aa.cloud.databricks.com/editor/queries/3638485123603774?o=519704688435860$0
# --- 2. Завантаження Spark DataFrames ---
spark_df_la = spark.table(table_la)
spark_df_ny = spark.table(table_ny)
    
# --- 3. Збагачення (Додавання стовпця 'city') ---
# Додаємо стовпець 'city', щоб розрізняти записи
spark_df_la = spark_df_la.withColumn("city", F.lit("Los Angeles"))
spark_df_ny = spark_df_ny.withColumn("city", F.lit("New York"))

# --- 4. Об'єднання (Union) ---
# Об'єднуємо два Spark DataFrames в один
# unionByName безпечніший, оскільки він з'єднує по іменах стовпців, а не по порядку
spark_df_all = spark_df_la.unionByName(spark_df_ny)

# --- 5. Конвертація в pandas ---
# Тепер конвертуємо ОДИН великий DataFrame в pandas
print("Завантаження та об'єднання... Це може зайняти деякий час.")
df_silver_all = spark_df_all.toPandas()
print("✅ Дані успішно завантажено в єдиний pandas DataFrame: df_silver_all")

# --- 6. Перевірка ---
print(f"\nЗагальна кількість записів: {len(df_silver_all)}")
print("\nРозподіл по містах:")
print(df_silver_all['city'].value_counts())

print("\n[ .head() ]")
print(df_silver_all.head())

print("\n[ .info() ]")
df_silver_all.info()

In [0]:
display(df_silver_all.head(5))

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Встановлюємо стиль для графіків
sns.set_theme(style="whitegrid")

# Припустимо, ваш об'єднаний DataFrame називається df_silver_all
# (як ми створили в попередньому кроці)

# Створимо копію для безпечного аналізу
df_analysis = df_silver_all.copy()

print("Бібліотеки імпортовано. DataFrame df_analysis готовий.")

In [0]:
# --- 1.1: Очищення та обробка ціни ---
# Ваш колега вже зробив це в Silver, але ми перевіримо
# Видаляємо рядки, де ціна NaN (якщо такі лишились)
df_analysis = df_analysis.dropna(subset=['price'])

# Фільтруємо екстремальні викиди, які роблять графіки нечитабельними 
# (Беремо 99-й перцентиль як верхню межу)
price_cap = df_analysis['price'].quantile(0.99)
df_analysis = df_analysis[df_analysis['price'] <= price_cap]
df_analysis = df_analysis[df_analysis['price'] > 0] # Видаляємо нульові, якщо є

# Створюємо log_price
df_analysis['log_price'] = np.log1p(df_analysis['price'])

print(f"Дані очищено. Використовуємо верхню межу ціни: ${price_cap:,.0f}")

# --- 1.2: Візуалізація: Розподіл 'log_price' ---
plt.figure(figsize=(16, 9))
sns.histplot(df_analysis, x='log_price', hue='city', kde=True, bins=50)
plt.title('Розподіл Log-Price (Ціни) для LA та NY', fontsize=16)
plt.xlabel('Log(Price + 1)')
plt.ylabel('Кількість оголошень')
plt.savefig('1_price_log_distribution.png')
print("✅ Збережено: 1_price_log_distribution.png")
plt.show()

# --- 1.3: Візуалізація: Порівняння цін за допомогою Boxplot ---
plt.figure(figsize=(16, 9))
sns.boxplot(data=df_analysis, x='city', y='price', palette='pastel')
plt.title('Порівняння "чистих" цін (до $%.0f) між LA та NY' % price_cap, fontsize=16)
plt.xlabel('Місто')
plt.ylabel('Ціна (USD)')
plt.savefig('2_price_boxplot_comparison.png')
print("✅ Збережено: 2_price_boxplot_comparison.png")
plt.show()

In [0]:
# --- 2.1: Візуалізація: Розподіл 'room_type' ---
plt.figure(figsize=(12, 7))
sns.countplot(data=df_analysis, x='room_type', hue='city', order=df_analysis['room_type'].value_counts().index)
plt.title('Популярність типів житла (Room Type) в LA та NY', fontsize=16)
plt.xlabel('Тип житла')
plt.ylabel('Кількість оголошень')
plt.savefig('3_room_type_distribution.png')
print("✅ Збережено: 3_room_type_distribution.png")
plt.show()

# --- 2.2: Аналіз: Топ-10 районів ('neighbourhood') ---
# Цей аналіз складніший, тому ми використовуємо subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 8), sharex=True)
fig.suptitle('Топ-10 районів (Neighbourhood) за кількістю оголошень', fontsize=20)

# LA
la_top_neigh = df_analysis[df_analysis['city'] == 'Los Angeles']['neighbourhood'].value_counts().head(10)
sns.barplot(ax=axes[0], y=la_top_neigh.index, x=la_top_neigh.values, palette='viridis')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Кількість оголошень')

# NY
ny_top_neigh = df_analysis[df_analysis['city'] == 'New York']['neighbourhood'].value_counts().head(10)
sns.barplot(ax=axes[1], y=ny_top_neigh.index, x=ny_top_neigh.values, palette='plasma')
axes[1].set_title('New York')
axes[1].set_xlabel('Кількість оголошень')

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Залишаємо місце для загального заголовку
plt.savefig('4_top_10_neighbourhoods.png')
print("✅ Збережено: 4_top_10_neighbourhoods.png")
plt.show()

In [0]:
import plotly.express as px

# Фільтруємо міста
df_la_geo = df_analysis[df_analysis['city'] == 'Los Angeles']
df_ny_geo = df_analysis[df_analysis['city'] == 'New York']

# LA
fig_la = px.scatter_mapbox(
    df_la_geo,
    lat="latitude",
    lon="longitude",
    color="log_price",
    size_max=5,
    zoom=9,
    height=500,
    mapbox_style="carto-positron"
)
fig_la.update_layout(title="Los Angeles — Housing Price Map")
fig_la.show()

# NY
fig_ny = px.scatter_mapbox(
    df_ny_geo,
    lat="latitude",
    lon="longitude",
    color="log_price",
    size_max=5,
    zoom=9,
    height=500,
    mapbox_style="carto-positron"
)
fig_ny.update_layout(title="New York — Housing Price Map")
fig_ny.show()


In [0]:
# --- 4.1: Очищення: 'review_scores_rating' ---
# Переконаємося, що немає пропусків, інакше графік зламається
df_analysis_ratings = df_analysis.dropna(subset=['review_scores_rating'])

# --- 4.2: Візуалізація: Розподіл 'review_scores_rating' ---
plt.figure(figsize=(12, 6))
sns.histplot(df_analysis_ratings, x='review_scores_rating', hue='city', kde=True, bins=30)
plt.title('Розподіл рейтингів (Review Scores Rating)', fontsize=16)
plt.xlabel('Рейтинг (1-5)')
plt.ylabel('Кількість оголошень')
plt.savefig('6_ratings_distribution.png')
print("✅ Збережено: 6_ratings_distribution.png")
plt.show()

# --- 4.3: Візуалізація: Залежність ціни від рейтингу ---
# Використовуємо regplot, щоб побачити лінію тренду
# Оскільки точок ду-у-же багато, візьмемо 10% вибірку (sample) для графіка
df_sample = df_analysis_ratings.sample(frac=0.1)

plt.figure(figsize=(12, 7))
sns.regplot(data=df_sample, x='review_scores_rating', y='log_price',
            scatter_kws={'alpha':0.1, 's':10}, line_kws={'color':'red'})
plt.title('Залежність Log-Price від Рейтингу', fontsize=16)
plt.xlabel('Рейтинг')
plt.ylabel('Log(Price + 1)')
plt.savefig('7_price_vs_rating_scatterplot.png')
print("✅ Збережено: 7_price_vs_rating_scatterplot.png")
plt.show()

In [0]:
# --- 5.1: Аналіз: 'calculated_host_listings_count' (РОЗРАХУНОК) ---
# KeyError означає, що стовпця немає. Давайте створимо його!

print("Стовпець 'calculated_host_listings_count' відсутній.")
print("Розрахунок вручну...")

# 1. Рахуємо кількість оголошень для кожного host_id
host_counts_map = df_analysis.groupby('host_id')['listing_id'].count()

# 2. Створюємо новий стовпець, "приклеюючи" цю кількість до кожного оголошення
# .map() - це дуже швидкий спосіб зробити це
df_analysis['calculated_host_listings_count'] = df_analysis['host_id'].map(host_counts_map)

print("✅ Стовпець 'calculated_host_listings_count' успішно створено.")

# --- 5.2: Візуалізація: Тип Хоста ---
# Тепер цей код має спрацювати
def host_type(count):
    if count == 1:
        return '1 (Single Listing)'
    elif 2 <= count <= 4:
        return '2-4 (Multi Listing)'
    else:
        return '5+ (Super Host/Company)'

df_analysis['host_listing_category'] = df_analysis['calculated_host_listings_count'].apply(host_type)

plt.figure(figsize=(12, 7))
sns.countplot(data=df_analysis, x='host_listing_category', hue='city',
              order=['1 (Single Listing)', '2-4 (Multi Listing)', '5+ (Super Host/Company)'])
plt.title('Тип Хоста (за кількістю оголошень)', fontsize=16)
plt.xlabel('Категорія Хоста')
plt.ylabel('Кількість оголошень')

# Використовуємо plt.show() для відображення в Databricks
plt.show()


# --- 5.3: Візуалізація: 'availability_365' (Доступність) ---
# Цей код, ймовірно, спрацює. Якщо ні, значить 'availability_365' також відсутній.
try:
    plt.figure(figsize=(12, 6))
    sns.histplot(df_analysis, x='availability_365', hue='city', bins=30, kde=False)
    plt.title('Розподіл доступності (Availability_365)', fontsize=16)
    plt.xlabel('Кількість доступних днів на рік')
    plt.ylabel('Кількість оголошень')
    
    # Використовуємо plt.show() для відображення в Databricks
    plt.show()

    print("\n--- 🏁 Етап 5 завершено! ---")

except KeyError as e:
    print(f"\n❌ ОЙ! Схоже, стовпець {e} також відсутній.")
    print("Повідомте про це вашого 'Data Engineer' (Учасника 1).")
    print("Цей графік побудувати неможливо без цього стовпця.")

In [0]:
# --- 6.1: Візуалізація: Ціна vs. Тип Житла vs. Місто ---
# Ми використовуємо 'log_price' для осі Y, оскільки 'price' має занадто великі викиди,
# які зроблять графік нечитабельним.
plt.figure(figsize=(15, 8))
sns.violinplot(data=df_analysis, x='room_type', y='log_price', hue='city', split=True, palette='pastel')
plt.title('Log-Price Distribution by Room Type and City', fontsize=16)
plt.xlabel('Room Type')
plt.ylabel('Log(Price + 1)')
plt.legend(title='City', loc='upper left')

# Використовуємо plt.show() для відображення в Databricks
plt.show()

In [0]:
# --- 6.1: Агрегація: Пошук найдорожчих районів ---
# Ми не можемо просто побудувати boxplot, бо у 'neighbourhood' тисячі значень.
# Давайте знайдемо медіанну ціну для кожного району.

# 1. Рахуємо медіанну ціну ТА кількість оголошень для кожного району
neigh_stats = df_analysis.groupby(['city', 'neighbourhood'])['price'].agg(
    median_price='median', 
    listing_count='count'
).reset_index()

# 2. Фільтруємо "шум" - райони, де менше 10 оголошень
neigh_stats_filtered = neigh_stats[neigh_stats['listing_count'] >= 10]

# --- 6.2: Візуалізація: Топ-20 найдорожчих районів ---
fig, axes = plt.subplots(1, 2, figsize=(16, 10))
fig.suptitle('Топ-20 Найдорожчих Районів (за Медіанною Ціною)', fontsize=20)

# LA
la_top_expensive = neigh_stats_filtered[neigh_stats_filtered['city'] == 'Los Angeles'].nlargest(20, 'median_price')
sns.barplot(ax=axes[0], data=la_top_expensive, y='neighbourhood', x='median_price', palette='Reds_r')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Медіанна Ціна (USD)')
axes[0].set_ylabel('Район')

# NY
ny_top_expensive = neigh_stats_filtered[neigh_stats_filtered['city'] == 'New York'].nlargest(20, 'median_price')
sns.barplot(ax=axes[1], data=ny_top_expensive, y='neighbourhood', x='median_price', palette='Blues_r')
axes[1].set_title('New York')
axes[1].set_xlabel('Медіанна Ціна (USD)')
axes[1].set_ylabel('') # Прибираємо дублікат осі Y

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [0]:
# --- 7.1: Підготовка даних для кореляції ---
# Вибираємо тільки ті стовпці, які ТОЧНО є у вашому DataFrame
numeric_cols = [
    'price', 
    'log_price',                  # Ми створили
    'number_of_reviews', 
    'review_scores_rating', 
    'latitude', 
    'longitude', 
    'calculated_host_listings_count' # Ми створили
]

# Спробуємо взяти тільки ті стовпці, які реально існують
available_numeric_cols = [col for col in numeric_cols if col in df_analysis.columns]

print(f"Стовпці, що використовуються для кореляції: {available_numeric_cols}")

# .dropna() тут критично
df_corr = df_analysis[available_numeric_cols].dropna()

# Рахуємо матрицю кореляцій
corr_matrix = df_corr.corr()

# --- 7.2: Візуалізація: Теплова карта кореляцій ---
plt.figure(figsize=(12, 9))
sns.heatmap(
    corr_matrix, 
    annot=True,     # Показувати цифри всередині клітинок
    fmt='.2f',      # Форматувати до 2-х знаків після коми
    cmap='coolwarm',# Кольорова схема
    vmin=-1,        # Встановити мінімум шкали
    vmax=1          # Встановити максимум шкали
)
plt.title('Теплова карта кореляцій (Тільки доступні дані)', fontsize=16)
plt.show()

In [0]:
# --- 8.1: Аналіз: Топ-20 найактивніших хостів ---
# Ми вже маємо стовпець 'calculated_host_listings_count'
# Давайте знайдемо хостів з найбільшою кількістю оголошень

# Видаляємо дублікати, щоб у нас був 1 рядок = 1 хост
df_hosts = df_analysis.drop_duplicates(subset=['host_id'])

fig, axes = plt.subplots(1, 2, figsize=(16, 10))
fig.suptitle('Топ-20 Хостів за Кількістю Оголошень', fontsize=20)

# LA
la_top_hosts = df_hosts[df_hosts['city'] == 'Los Angeles'].nlargest(20, 'calculated_host_listings_count')
sns.barplot(ax=axes[0], data=la_top_hosts, y='host_name', x='calculated_host_listings_count', palette='Oranges_r')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Кількість Оголошень')
axes[0].set_ylabel('Ім\'я Хоста')

# NY
ny_top_hosts = df_hosts[df_hosts['city'] == 'New York'].nlargest(20, 'calculated_host_listings_count')
sns.barplot(ax=axes[1], data=ny_top_hosts, y='host_name', x='calculated_host_listings_count', palette='Greens_r')
axes[1].set_title('New York')
axes[1].set_xlabel('Кількість Оголошень')
axes[1].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


# --- 8.2: Візуалізація: Рейтинг vs. Кількість відгуків ---
# Чи стає рейтинг кращим/гіршим з часом (з кількістю відгуків)?

# Створимо 'log_reviews' для кращої візуалізації
df_analysis['log_reviews'] = np.log1p(df_analysis['number_of_reviews'])
df_analysis_ratings = df_analysis.dropna(subset=['review_scores_rating']) # Видаляємо NaN в рейтингах

# Беремо вибірку (sample), бо точок ду-у-же багато
df_sample_ratings = df_analysis_ratings.sample(frac=0.1)

sns.lmplot(
    data=df_sample_ratings, 
    x='log_reviews', 
    y='review_scores_rating', 
    hue='city', 
    height=6, 
    aspect=1.5, 
    scatter_kws={'alpha':0.1}, # Робимо точки прозорими
    line_kws={'lw': 2}         # Робимо лінію тренду товстішою
)
plt.title('Залежність Рейтингу від К-сті Відгуків', fontsize=16)
plt.xlabel('Log(Number of Reviews + 1)')
plt.ylabel('Review Score Rating (1-5)')
plt.show()

In [0]:
# --- 9.1: Створення категорії "Наявність відгуків" ---
def review_status(review_count):
    if review_count == 0:
        return '0 (No Reviews)'
    else:
        return '1+ (Has Reviews)'

df_analysis['review_status'] = df_analysis['number_of_reviews'].apply(review_status)

print("Розподіл оголошень за наявністю відгуків:")
print(df_analysis['review_status'].value_counts())

# --- 9.2: Візуалізація: Log-Price (ціна) vs. Наявність відгуків ---
plt.figure(figsize=(10, 7))
sns.boxplot(
    data=df_analysis, 
    x='review_status', 
    y='log_price', 
    hue='city',
    order=['0 (No Reviews)', '1+ (Has Reviews)'] # Встановлюємо порядок
)
plt.title('Log-Price (Ціна) для "Нових" (0 відгуків) vs. "Встановлених" (1+ відгук)', fontsize=16)
plt.xlabel('Статус відгуків')
plt.ylabel('Log(Price + 1)')
plt.show()

# --- 9.3: Візуалізація: Ціна vs. Рейтинг (по-іншому) ---
# 'jointplot' дозволяє побачити гістограми обох змінних
# Це покаже, чи є скупчення дорогих, але низькорейтингових оголошень
df_sample_ratings = df_analysis.dropna(subset=['review_scores_rating']).sample(frac=0.1)

sns.jointplot(
    data=df_sample_ratings, 
    x='review_scores_rating', 
    y='log_price', 
    kind='hex', # 'hex' (шестикутники) чудово показує щільність
    height=8,
    cmap='viridis'
)
plt.suptitle('Щільність оголошень: Log-Price vs. Rating', y=1.02, fontsize=16)
plt.xlabel('Review Score Rating')
plt.ylabel('Log(Price + 1)')
plt.tight_layout()
plt.show()

In [0]:
# --- 10.1: Перевірка/Створення 'host_listing_category' ---
# (Просто про всяк випадок, якщо ноутбук перезапускався)
if 'host_listing_category' not in df_analysis.columns:
    print("Створення 'host_listing_category'...")
    
    # Розраховуємо 'calculated_host_listings_count'
    if 'calculated_host_listings_count' not in df_analysis.columns:
        host_counts_map = df_analysis.groupby('host_id')['listing_id'].count()
        df_analysis['calculated_host_listings_count'] = df_analysis['host_id'].map(host_counts_map)

    # Функція для категорій
    def host_type(count):
        if count == 1:
            return '1 (Single Listing)'
        elif 2 <= count <= 4:
            return '2-4 (Multi Listing)'
        else:
            return '5+ (Super Host/Company)'
    
    df_analysis['host_listing_category'] = df_analysis['calculated_host_listings_count'].apply(host_type)
    print("...Створено.")

# --- 10.2: Візуалізація: Log-Price (ціна) vs. Тип Хоста ---
# 'violinplot' - це як boxplot, але показує ще й розподіл
plt.figure(figsize=(14, 8))
sns.violinplot(
    data=df_analysis, 
    x='host_listing_category', 
    y='log_price', 
    hue='city',
    order=['1 (Single Listing)', '2-4 (Multi Listing)', '5+ (Super Host/Company)'],
    split=True # Розділяє скрипки для LA/NY
)
plt.title('Log-Price (Ціна) vs. Тип Хоста', fontsize=16)
plt.xlabel('Категорія Хоста (за к-стю оголошень)')
plt.ylabel('Log(Price + 1)')
plt.show()

In [0]:
# --- 11.1: Фільтрація: Пошук оголошень з низьким рейтингом ---
rating_threshold = 3.5
df_low_ratings = df_analysis[
    (df_analysis['review_scores_rating'] <= rating_threshold) &
    (df_analysis['number_of_reviews'] >= 5) # Беремо ті, де є хоча б 5 відгуків
]

print(f"Знайдено {len(df_low_ratings)} оголошень з рейтингом <= {rating_threshold} (та >= 5 відгуками)")

# --- 11.2: Візуалізація: Гео-карта "Проблемних" Оголошень ---
# Це покаже, чи є "кластери" поганого сервісу
fig, axes = plt.subplots(1, 2, figsize=(16, 10), sharey=True)
fig.suptitle(f'Гео-карта оголошень з рейтингом ≤ {rating_threshold}', fontsize=20)

# LA
df_la_low = df_low_ratings[df_low_ratings['city'] == 'Los Angeles']
sns.scatterplot(ax=axes[0], data=df_la_low, x='longitude', y='latitude', 
                hue='review_scores_rating', # Колір за рейтингом
                palette='autumn_r', size=10, legend=True)
axes[0].set_title(f'Los Angeles ({len(df_la_low)} оголошень)')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
# Накладаємо "фон" з усіх оголошень для контексту
sns.scatterplot(ax=axes[0], data=df_analysis[df_analysis['city'] == 'Los Angeles'], 
                x='longitude', y='latitude', color='gray', alpha=0.05, s=2, legend=False)


# NY
df_ny_low = df_low_ratings[df_low_ratings['city'] == 'New York']
sns.scatterplot(ax=axes[1], data=df_ny_low, x='longitude', y='latitude', 
                hue='review_scores_rating', # Колір за рейтингом
                palette='autumn_r', size=10, legend=True)
axes[1].set_title(f'New York ({len(df_ny_low)} оголошень)')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('')
# Накладаємо "фон" з усіх оголошень для контексту
sns.scatterplot(ax=axes[1], data=df_analysis[df_analysis['city'] == 'New York'], 
                x='longitude', y='latitude', color='gray', alpha=0.05, s=2, legend=False)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()